## Interactive lammps

In [ ]:
from lammps import PyLammps

In [ ]:
L = PyLammps()

####  SET SIMULATION CONDITIONS

In [ ]:
L.command("units lj")
L.command("atom_style full")
L.command("pair_style lj/cut 2.5")
L.command("bond_style harmonic")
L.command("special_bonds fene")

L.command("dimension 3")
L.command("boundary p p p")
L.command("neighbor 0.8 bin")

In [ ]:
L.command('read_data lipit.data')

In [ ]:
L.command('change_box all boundary p p f')

In [ ]:
L.command('change_box all z delta -40.0 20.0 units box')

####  DEFINE VARIABLES

In [ ]:
L.command('variable velocityBullet equal -2')
L.command('variable radiusBullet equal 5')
L.command('variable hBullet equal 5+${radiusBullet}')

#### CHOOSE THERMODYNAMIC OUTPUT

In [ ]:
L.command('thermo  200')
L.command('thermo_style    custom step temp press ebond epair ke')

#### CLAMP BOUNDARIES OF THE FILM

In [ ]:
L.command('region hole cylinder z 0 0 20 -10.0 10.0 side in units box')
L.command('group hole region hole')
L.command('group film type 1 2')
L.command('group base subtract film hole')

L.command('fix nailed base setforce 0.0 0.0 0.0')
L.command('velocity base set 0.0 0.0 0.0 units box')
L.command('velocity hole create 0.1 239472')

#### CREATE BULLET ABOVE THE FILM

In [ ]:
L.command('lattice diamond 7.2')
L.command('region 1 sphere 0 0 ${hBullet} ${radiusBullet} side in units box')
L.command('create_atoms 3 region 1 units box')
L.command('group indent type 3')
L.command('neigh_modify exclude group indent indent')
L.command('fix 1 indent rigid single')
L.command('velocity indent set 0.0 0.0 -2 units box')

#### SHOOT!

In [ ]:
L.command('fix nve1 hole nve')
L.command('compute stressA all stress/atom NULL')
L.command('compute 1 indent ke')
L.command('fix kin all ave/time 1 1 10 c_1 file kinetic.data')
L.command('dump mydump all custom 200 Conf.dat id type x y z c_stressA[1] c_stressA[2] c_stressA[3]')
L.command('dump_modify mydump sort id')
L.command('run 8000')

### Visualisation

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
import nglview as nv
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
u = mda.Universe('Conf.dat',format='LAMMPSDUMP')
atoms = u.atoms

In [ ]:
view = nv.show_mdanalysis(atoms)

In [ ]:
view

In [ ]:
kin = np.loadtxt('kinetic.data')
time = 0.005*kin[:,0]
vel = np.sqrt(2*kin[:,1]/3760.0)

plt.figure(dpi=100)
plt.plot(time,vel);
plt.xlabel('Time');
plt.ylabel("Velocity of the bullet");